# Introduction:
I am trying to predict whether on not someone will have heart disease based on serveral factors related to their current health and other heart realted issues. The datset that i am i using is the predicting heart failure dataset. I predict that a random forest model will be the best and the most accurate model to use to make accurate predictions. I believe that my strongest feature to affect predictions will be the age of a person. Regarding the age feature i predict people in a higher age bracket will be more prone to heart failure. My goal for the accuracy of my predictions is to achieve moderatly accurate predicitions while ensuring that i do not overift my data which i can avoid by spliting my data into training and testing sets.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=Warning)
from sklearn.tree import DecisionTreeClassifier # Our model

#Input data files are available in the read-only "../input/" directory

#Lists all files in input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Explore the data
To import the data we need to go into the heart database, we have 14 different categories in the heart database, these catergories are 'age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'. 
These categories represent: age - age, sex - gender, cp - any chest pain and what level of chest pain, trtbps - resting blood pressure (in mg), chol - cholestoral in mg/dl fetched via BMI sensor, fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false), restecg - resting electrocardiographic results, thalachh - maximum heart rate achieved, exng - exercise induced angina (1 = yes; 0 = no), oldpeak - previous peak, slp - slope, caa - number of major vessels in subject (1-3), thall - thal rate, output - If someone had heart disease or not (target variable). For output 0 means that the person has heart failure and 1 means they do not.

It is also important to note that there are 303 different values per column

In [ ]:

train_file_path = '../input/heart-attack-analysis-prediction-dataset/heart.csv'
# Create a new Pandas DataFrame with our training data
heart_train_data = pd.read_csv(train_file_path)


print(heart_train_data.columns) #shows all of the columns 
#heart_train_data.describe(include='all')

In [ ]:
train_dataset = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
print(train_dataset.head())
#print(dataset.shape)
print(train_dataset.describe(include = 'all'))
#print(dataset.info())
train_dataset.isnull().values.any() # checks if there are any zero "null" values
#pd.value_counts(dataset['output'])
#print("--------------------------------------------")
#saturation_dataset = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
#print(saturation_dataset.describe(include = 'all'))

I am predicting the output variable which is read as 0 or 1 (0 is heart disease, 1 is no heart disease). A suitable prediction target is to be able to use a model that can predict test data with above an accuracy of half of the data.

# Graphs to visualise and represent data

In [ ]:
sns.displot(train_dataset["thalachh"], color="blue"); #This graph explores the trend of the feature thalachh 
plt.title("DISTRIBUTION OF HEART RATE AMONG PATIENTS", fontsize=18)
plt.xlabel("HEART RATE", fontsize=20)
plt.ylabel("COUNT", fontsize=20)
plt.show();

In [ ]:
plt.figure(figsize=(13, 10))
matrix = np.triu(train_dataset.corr()) #compares the corralation of every variable to each other 
sns.heatmap(train_dataset.corr(), annot=True,
            linewidth=.8, mask=matrix, cmap="rocket");
#finds the corralation coefficient inbetween every variable.

This table shows the correlation inbetween each of the variables, so we can see which variables have high correlation with other variables in order to determine the best 'starting' variables and also use a visual representation of what variables have a high correlation with each other.

In [ ]:
plt.figure(figsize = (10, 5))
plt.style.use("ggplot")
sns.countplot(x = train_dataset["age"]);  # using countplot
plt.title("Age compared to #people with heart failure", fontsize=20)
plt.xlabel("AGE", fontsize = 15)
plt.ylabel("#HEART FAILURE CASES ", fontsize=15)
plt.show()

This graph shows the correlatioon between age and the number of heart failure cases. You can see in this graph that people who are aged above 40 and below 68 are more likely to get heart attack. People at the age of 58 are highly prone to heart attacks. We also see a signifact dip in the range of  46 - 50 which could be attributed to other external factors suggesting some inconsistencies or outliers in the data. Although it is hard to clarify what can be defined as an outlier for this data as there are only two possible outcomes, heart failure or no heart failure.

In [ ]:
plt.figure(figsize = (12, 8))
plt.style.use("ggplot")
sns.histplot(data = train_dataset, x = 'age', hue = 'output') #1 = heart failure (blue), 0 = no heart failure (pink)
plt.title("AGE EFFECT ON THE HEART-ATTACK")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

This graph shows the correlation between age and heart failure (0 = heart failure, 1 = no heart failure), we see a significant increase around the age of 60 and then a slow decline.

# Preparing the Data
We want to predict if someone is likely to have a heart attack. To do this it would be logical to choose the most relevent feeatures and model our data of this. However we do not want to get rid of any features as none can be classified as needless or un-neccesary as all data contributes to if the person has a heart attack or not, just at varying levels. 

There is no categorical data and there are no null or missing values in my data so i dont need to drop anything. 

As every feature plays a role in deducing if someone undergoes heart failure, we cannot get rid of a feature. This also applies to data points, this particular data set has limited data (only 303) whcih means that it is hard to clarify what can be defined as an outlier for this data as every piece of data contributes to finding if someone has heart failure or not and the only data that can be deemed an outlier is data that completely contradicts with the majority of the other data. Which in the case of this dataset, there are no such cases.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create our new training set containing only the features we want
features = ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh','exng', 'oldpeak', 'slp', 'caa', 'thall', 'output']
prepared_data = train_dataset[features]

# Drop rows that contain missing values
prepared_data = prepared_data.dropna(axis = 0)

# Check that I still have a good 'count' value.
prepared_data.describe(include = 'all')
prepared_data.head()

In [ ]:

#model = DecisionTreeRegressor(random_state=1)
y = prepared_data.output
# Drop the Survived column (axis=1 indicates column)
X = prepared_data.drop('output', axis=1)
print(X.head())
print(y)
#print(X.describe(include = 'all'))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1) #Splitting the data into training and validation data

#STANDARDISING THE DATA
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #Training data
X_test = sc.transform(X_test) #Validation data

model = DecisionTreeRegressor(random_state=1)
#fit model
model.fit(X_train, y_train)

Spliting and training data + descision tree

I have split the data and then standardised it so that i can test it using my models. I have split the data so that the validation set is much larger then my training data which gives me a broder range to compare my data with. Having less data and therfore less training data will negativily impact my models performance however as it will have a smaller collection of data to compare with which means that its predictions will be less accurate then if i had a wider spread of data. This model is a decision tree regressor which makes reasonably accurate predictions based on what values it gets for its data which will make descisions down a 'tree' to come up with a final answer whether someone has heart disease or not. 

# Training a Model and Making Predictions
We can see that my MAE for the testing or validation data is 0.276315... and my MAE for the training data is 0. This is because the data that i have used for my training data is exactly the same data that i am making predictions with meaning that the training data is overfit. My MAE for the validation data however is 0.276315 which isn't too high and suggests that the descision tree model is a reasonably good and also efficent way to develop accurate predcitions about heart disease. 

In [ ]:
print("The predictions for validation data:")
print(model.predict(X_test))
print("The predictions for training data:")
print(model.predict(X_train))
#test_predictions = model.predict(X_test)
#print("Validation MAE {:,.0f}".format(test_mae)) doesn't work as MAE has decimal places
#print(y_train)
#print(model.predict(X))
#print(model.predict(X.head()))


test_predictions = model.predict(X_test)
test_mae = mean_absolute_error(test_predictions, y_test)
#This shows the MAE of the X validation  data and the y test data
print("MAE of testing data: "  + str(test_mae))

train_predictions = model.predict(X_train)
train_mae = mean_absolute_error(y_train, train_predictions)
##This shows the MAE of the X training data and the y training data, this predicts 0 as we used X and y training data for our model.
print("MAE of training data: " + str(train_mae))

We achieve a MAE of 0.276315 on our validation data with our model. We have achieved this value by modeling our training data and testing it with the validation data. What if I used my validation data to model my training data instead to try to achieve a better MAE.

In [ ]:
reversed_model = DecisionTreeRegressor(random_state=1)
#fit model
reversed_model.fit(X_test, y_test) 

print("The predictions for old validation data (now training data):")
print(reversed_model.predict(X_test))
print("The predictions for the old training data (now validation data):")
print(reversed_model.predict(X_train))

test_predictions_2 = reversed_model.predict(X_test)
test_mae_2 = mean_absolute_error(test_predictions_2, y_test)
#This shows the MAE of the X validation  data and the y test data this predicts 0 as we used X and y validation data for our model.
print("MAE of old validation new training data: "  + str(test_mae_2))

train_predictions_2 = reversed_model.predict(X_train)
train_mae_2 = mean_absolute_error(y_train, train_predictions_2)
##This shows the MAE of the X training data and the y training data
print("MAE of old training new validation data: " + str(train_mae_2))

The MAE we get for the validation data in our first model is aprox 0.276, where the MAE for our validation data in our second model is aprox 0.295 which is a greater MAE which means that using our original training data model to predict our original validation data, is better then using the inverted model.

# Hyperparamater tuning 
How does adding differing max numbers of leaf nodes affect our results?

In [ ]:
#100 max leaf node model 
hundredleaf_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
hundredleaf_model.fit(X_train, y_train)
hundred_leaf_predictions = model.predict(X_test)
hundred_leaf_mae = mean_absolute_error(hundred_leaf_predictions, y_test)
print("MAE of a hundred leaf node model validation data: " + str(hundred_leaf_mae))

#50 max leaf node model 
fiftyleaf_model = DecisionTreeRegressor(max_leaf_nodes=50, random_state=1)
fiftyleaf_model.fit(X_train, y_train)
fifty_leaf_predictions = model.predict(X_test)
fifty_leaf_mae = mean_absolute_error(fifty_leaf_predictions, y_test)
print("MAE of a fifty leaf node model validation data: " + str(fifty_leaf_mae))

#20 max leaf node model 
twentyleaf_model = DecisionTreeRegressor(max_leaf_nodes=20, random_state=1)
twentyleaf_model.fit(X_train, y_train)
twenty_leaf_predictions = model.predict(X_test)
twenty_leaf_mae = mean_absolute_error(twenty_leaf_predictions, y_test)
print("MAE of a twenty leaf node model validation data: " + str(twenty_leaf_mae))

#10 max leaf node model 
tenleaf_model = DecisionTreeRegressor(max_leaf_nodes=10, random_state=1)
tenleaf_model.fit(X_train, y_train)
ten_leaf_predictions = model.predict(X_test)
ten_leaf_mae = mean_absolute_error(ten_leaf_predictions, y_test)
print("MAE of a ten leaf node model validation data: " + str(ten_leaf_mae))



All results appear the same. Tuning this paramater seems to have no visible effect on the data and therefore has no affect on the accuracy.

In [ ]:
from sklearn.tree import DecisionTreeClassifier ,plot_tree
heart_predictor = DecisionTreeClassifier(max_depth = 4)
heart_predictor.fit(X, y)

plt.figure(figsize = (30,20))
plot_tree(heart_predictor,
          feature_names = X.columns,
          class_names =['heart disease', 'no heart disease'],
          filled=True)
plt.show()


This decision tree classifier gives a visual example of what is happening inside of my descision tree when it decides if someone has heart disease or not based on the feature values it is given. We can see the gini impurity in this descision tree classifier.

In [ ]:
print("Making predictions for the first 5 people in the training set.")
print("The predictions are:")

print(heart_predictor.predict(X.head())) #predictions of who has heart disease

print("Compared to:")
print(y.head()) #data of heart disease 


We can see that out of the first few people in the training set i predicted 4 out of 5 people correctly. This suggests that my descision tree classifier isn't overfit to the data due to me splitting my data, but my predicitons aren't perfect and could probably be improved with a different model.  

# RANDOM FOREST
A random forest model is made up of a large number of small decision trees, called estimators, which each produce their own predictions. These small predicitons all add up to make one final prediction at the end. A random forest can be seen as a more accurate descision tree. We can see that i achieved a 0 MAE for my random forest model which i believe can be attributed to the randomness of the random forest and the fact that there is limited datapoints to measure with.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train, y_train)
rf_test_predictions = rf_model.predict(X_test)
rf_test_mae = mean_absolute_error(rf_test_predictions, y_test)
print("Validation MAE for Random Forest Model: " + str(rf_test_mae))

# CONCLUSION
The purpose of the investigation was to deduce if someone had heart disease or not based on certain features of their health. I predicted whether or not someone had heart disease using a descision tree by splitting and standardising the data. The quality of my predictions with the descision tree were reasonable. The descision tree was the best model and it usually made reliable and accurate predictions and showed that it didn't overfit or underfit to the data. I achieved my goal of not wanting to overfit my data and creating semi-accurate predictions,however i hypothesised that a random forest model would be the best model to use to make accurate predictions which was wrong as it had a higher MAE the the descision tree regressor. I believed that my strongest feature that would affect predictions would be the age of a person which proved to be true with the highest corration to the results in the output. My hyperparameter tuning had very little effect on my data and changing the output it gave so it was uneffective. Other models could have been used to a better effect if this dataset was bigger as this dataset has only 300 results to train your model and then also predict with your model which isn't enough to create a reliable large scale model.